In [1]:
import pandas as pd
import os
from pathlib import Path
from getpass import getpass

# Read in data from CSV. 
Data originally downloaded from https://www.kaggle.com/datasets/tuhinmallick02/insider-trading-sp-500

The data represents stock transactions by insiders at corporations.

In [2]:
insider_df = pd.read_csv("insiderdata.csv")

In [3]:
insider_df.head()

,Unnamed: 0.1,Unnamed: 0,Buy/Sell,Transaction Date,Acceptance Date Time,Issuer Name,Issuer Trading Symbol,Reporting Owner Name,Reporting Owner Relationship,Transcation Share,Price Per share,Total Value,Shares Owned Following Transaction,Form,sector,market_cap
0,0,0,Sell,2017-01-04,2017-01-06 20:57:22,Immune Design Corp.,IMDZ,Brady Stephen R,officer,"1,642",$5.3500,"$8,784.7000","30,490",Form 4,Health Care,282403939.76
1,1,1,Sell,2017-01-05,2017-01-06 20:46:33,HURCO COMPANIES INC,HURC,Volovic Gregory S,officer,"2,966",$33.9000,"$100,547.0000","28,839",Form 4,Capital Goods,279583548.51
2,2,2,Sell,2017-01-06,2017-01-06 20:46:02,HURCO COMPANIES INC,HURC,McClelland Sonja K,officer,234,$33.7000,"$7,885.8000","17,353",Form 4,Capital Goods,279583548.51
3,3,3,Sell,2017-01-04,2017-01-06 20:46:02,HURCO COMPANIES INC,HURC,McClelland Sonja K,officer,372,$34.3000,"$12,759.6000","12,836",Form 4,Capital Goods,279583548.51
4,4,4,Sell,2017-01-05,2017-01-06 20:45:30,HURCO COMPANIES INC,HURC,Donlon John P.,officer,"1,018",$33.9000,"$34,510.2000","12,689",Form 4,Capital Goods,279583548.51


# Format data
Before we can use `neo4j-admin import`, we need to generate CSV files in the special format that the importer requires.

Rename columns to follow Neo4j property naming conventions.

In [4]:
rename_dict = {'Unnamed: 0':'transactionId', 
               'Buy/Sell':'transactionType', 
               'Transaction Date': 'transactionDate',
               'Acceptance Date Time': 'acceptanceDateTime', 
               'Issuer Name': 'issuerName', 
               'Issuer Trading Symbol': 'issuerTradingSymbol',
               'Reporting Owner Name': 'reportingOwnerName', 
               'Reporting Owner Relationship': 'reportingOwnerRelationship',
               'Transcation Share': 'transactionShare', 
               'Price Per share': 'pricePerShare', 
               'Total Value': 'totalValue',
               'Shares Owned Following Transaction': 'sharesOwnedFollowingTransaction', 
               'Form': 'form', 
               'sector': 'sector', 
               'market_cap': 'marketCap'}

In [5]:
insider_df.rename(rename_dict, axis=1, inplace=True)
insider_df.drop("Unnamed: 0.1", axis=1, inplace=True)
insider_df.head()

,transactionId,transactionType,transactionDate,acceptanceDateTime,issuerName,issuerTradingSymbol,reportingOwnerName,reportingOwnerRelationship,transactionShare,pricePerShare,totalValue,sharesOwnedFollowingTransaction,form,sector,marketCap
0,0,Sell,2017-01-04,2017-01-06 20:57:22,Immune Design Corp.,IMDZ,Brady Stephen R,officer,"1,642",$5.3500,"$8,784.7000","30,490",Form 4,Health Care,282403939.76
1,1,Sell,2017-01-05,2017-01-06 20:46:33,HURCO COMPANIES INC,HURC,Volovic Gregory S,officer,"2,966",$33.9000,"$100,547.0000","28,839",Form 4,Capital Goods,279583548.51
2,2,Sell,2017-01-06,2017-01-06 20:46:02,HURCO COMPANIES INC,HURC,McClelland Sonja K,officer,234,$33.7000,"$7,885.8000","17,353",Form 4,Capital Goods,279583548.51
3,3,Sell,2017-01-04,2017-01-06 20:46:02,HURCO COMPANIES INC,HURC,McClelland Sonja K,officer,372,$34.3000,"$12,759.6000","12,836",Form 4,Capital Goods,279583548.51
4,4,Sell,2017-01-05,2017-01-06 20:45:30,HURCO COMPANIES INC,HURC,Donlon John P.,officer,"1,018",$33.9000,"$34,510.2000","12,689",Form 4,Capital Goods,279583548.51


Split the dataframe into two parts: one part for demonstrating a full load, and a second part for demonstrating an incremental load.

In [6]:
full_df = insider_df.iloc[:20000, :]
incremental_df = insider_df.iloc[20000:, :]

# Create node files
## Companies

`neo4j-admin import` expects columns in the CSV to have special names. 
* A column name followed by colon and data type allows you to specify a data type other than string
* A column name followed by colon and ID identifies the unique identifier for the node. The string in parenthesis following the ID is the namespace for the node ID. Each ID must be unique within the namespace.

In [7]:
def prep_companies(data_df):
    company_df = data_df.loc[:, ['issuerName', 'issuerTradingSymbol', 'form', 'sector', 'marketCap']]
    company_df.drop_duplicates(subset = 'issuerTradingSymbol', keep='first', inplace=True)
    company_df['marketCap'] = pd.to_numeric(company_df['marketCap'], errors='coerce')
    company_df.columns = ['issuerName', 'issuerTradingSymbol:ID(Company)', 'form', 'sector', 'marketCap:double']
    company_df[':LABEL'] = 'Company'
    return company_df

In [8]:
full_company_df = prep_companies(full_df)
incremental_company_df = prep_companies(incremental_df)

We don't want to reimport companies that were already loaded as part of the full import. `neo4j-admin import` cannot update properties on existing nodes, so there is no reason to reload the existing nodes. The presence of previously loaded nodes will cause the incremental update to fail unless you set `--skip-duplicate-nodes=true`. We will remove these duplicate nodes from the import to avoid that problem.

When doing an incremental import, you have to have a unique constraint on the ID property. Appending `{label:Company}` at the end of the column name for the ID column tells `neo4j-admin import` which label to look for the unique constraint on. The importer can't get this information from the `:LABEL` column, because a node might have more than one label.

In [9]:
incremental_company_df = (
    incremental_company_df.loc[~incremental_company_df['issuerTradingSymbol:ID(Company)']
                               .isin(full_company_df['issuerTradingSymbol:ID(Company)'])])
incremental_company_df.rename({'issuerTradingSymbol:ID(Company)':'issuerTradingSymbol:ID(Company){label:Company}'}, 
                              axis=1, inplace=True)

In [10]:
incremental_company_df.head()

,issuerName,issuerTradingSymbol:ID(Company){label:Company},form,sector,marketCap:double,:LABEL
20067,"EQT Midstream Partners, LP",EQM,Form 4,Public Utilities,8.937275e+09,Company
20086,"WES Consulting, Inc.",WSCU,Form 4,unknown,NaN,Company
20109,CGI HOLDING CORP,CGIH,Form 4,unknown,NaN,Company
20147,PACIFIC CONTINENTAL CORP,PCBK,Form 4,unknown,NaN,Company
20194,ENERGIZER HOLDINGS INC,ENR,Form 4,Miscellaneous,3.206549e+09,Company


In [11]:
full_company_df.to_csv("./data/full/company.csv", index=False)
incremental_company_df.to_csv("./data/incremental/company.csv", index=False)

## Owners

In [12]:
def prep_owners(data_df):
    owner_df = data_df.loc[:, ['reportingOwnerName']].drop_duplicates()
    owner_df.columns = ['reportingOwnerName:ID(Owner)']
    owner_df[":LABEL"] = "Owner"
    return owner_df

In [13]:
full_owner_df = prep_owners(full_df)
incremental_owner_df = prep_owners(incremental_df)

In [14]:
incremental_owner_df = (
    incremental_owner_df.loc[~incremental_owner_df['reportingOwnerName:ID(Owner)']
                               .isin(full_owner_df['reportingOwnerName:ID(Owner)'])])
incremental_owner_df.rename({'reportingOwnerName:ID(Owner)': 'reportingOwnerName:ID(Owner){label:Owner}'}, 
                            axis=1, inplace=True)

In [15]:
full_owner_df.to_csv("./data/full/owner.csv", index=False)
incremental_owner_df.to_csv("./data/incremental/owner.csv", index=False)

## Transactions

The transaction data requires some manipulation to format the data for loading to Neo4j.

In [16]:
def currency_to_float(currency):
    currency = currency.replace("$", "")
    currency = currency.replace(",", "")
    currency = float(currency)
    return currency

def prep_transactions(data_df):
    transaction_df = data_df.loc[:, ['transactionId', 'transactionType', 'transactionDate',
                                     'acceptanceDateTime', 'reportingOwnerRelationship', 'transactionShare',
                                     'pricePerShare', 'totalValue', 'sharesOwnedFollowingTransaction']]
    transaction_df['acceptanceDateTime'] = transaction_df['acceptanceDateTime'].str.split(" ").str.join("T")
    numeric_df = transaction_df.loc[:,['transactionShare', 'pricePerShare', 'totalValue', 
                                       'sharesOwnedFollowingTransaction']].applymap(currency_to_float)
    transaction_df[['transactionShare', 'pricePerShare', 'totalValue', 
                    'sharesOwnedFollowingTransaction']] = numeric_df
    transaction_df.columns = ['transactionId:ID(Transaction)', 'transactionType', 'transactionDate:date', 
                              'acceptanceDateTime:datetime', 'reportingOwnerRelationship', 'transactionShare:double', 
                              'pricePerShare:double', 'totalValue:double', 'sharesOwnedFollowingTransaction:double']
    transaction_df[':LABEL'] = "Transaction"
    return transaction_df

In [17]:
full_transaction_df = prep_transactions(full_df)
incremental_transaction_df = prep_transactions(incremental_df)

In [18]:
incremental_transaction_df = (
    incremental_transaction_df.loc[~incremental_transaction_df['transactionId:ID(Transaction)']
                               .isin(full_transaction_df['transactionId:ID(Transaction)'])])
incremental_transaction_df.rename({'transactionId:ID(Transaction)':
                                   'transactionId:ID(Transaction){label:Transaction}'}, 
                                  axis=1, inplace=True)

In [19]:
incremental_transaction_df.head()

,transactionId:ID(Transaction){label:Transaction},transactionType,transactionDate:date,acceptanceDateTime:datetime,reportingOwnerRelationship,transactionShare:double,pricePerShare:double,totalValue:double,sharesOwnedFollowingTransaction:double,:LABEL
20000,20000,Sell,2016-11-15,2016-11-16T18:08:54,officer,249.0,10.7113,2667.11,246296.0,Transaction
20001,20001,Sell,2016-11-16,2016-11-16T18:07:11,officer,10000.0,133.2540,1332540.00,379896.0,Transaction
20002,20002,Buy,2016-11-16,2016-11-16T18:04:57,officer,1950.0,25.9900,50680.50,16121.0,Transaction
20003,20003,Buy,2016-11-14,2016-11-16T18:04:56,officer,12000.0,18.0500,216600.00,38486.0,Transaction
20004,20004,Sell,2016-11-14,2016-11-16T18:04:56,officer,114865.0,57.7500,6633450.00,15236.0,Transaction


In [20]:
full_transaction_df.to_csv("./data/full/transaction.csv", index=False)
incremental_transaction_df.to_csv("./data/incremental/transaction.csv", index=False)

# Relationships
## MADE_TRANSACTION

The word in parenthesis after `:START_ID` and `:END_ID` tells `neo4j-admin import` which namespace to look up the ID in.

In [21]:
def prep_made_transaction(data_df):
    made_transaction_df = data_df.loc[:, ["transactionId", "reportingOwnerName"]]
    made_transaction_df.columns = [':END_ID(Transaction)',':START_ID(Owner)']
    made_transaction_df[':TYPE'] = "MADE_TRANSACTION"
    return made_transaction_df

In [22]:
full_made_transaction_df = prep_made_transaction(full_df)
incremental_made_transaction_df = prep_made_transaction(incremental_df)

In [23]:
full_made_transaction_df.head()

,:END_ID(Transaction),:START_ID(Owner),:TYPE
0,0,Brady Stephen R,MADE_TRANSACTION
1,1,Volovic Gregory S,MADE_TRANSACTION
2,2,McClelland Sonja K,MADE_TRANSACTION
3,3,McClelland Sonja K,MADE_TRANSACTION
4,4,Donlon John P.,MADE_TRANSACTION


In [24]:
full_made_transaction_df.to_csv("./data/full/made_transaction.csv", index=False)
incremental_made_transaction_df.to_csv("./data/incremental/made_transaction.csv", index=False)

## FOR_COMPANY

In [25]:
def prep_for_company(data_df):
    for_company_df = data_df.loc[:, ["transactionId", "issuerTradingSymbol"]]
    for_company_df.columns = [':START_ID(Transaction)', ':END_ID(Company)']
    for_company_df[':TYPE'] = "FOR_COMPANY"
    return for_company_df

In [26]:
full_for_company_df = prep_for_company(full_df)
incremental_for_company_df = prep_for_company(incremental_df)

In [27]:
full_for_company_df.head()

,:START_ID(Transaction),:END_ID(Company),:TYPE
0,0,IMDZ,FOR_COMPANY
1,1,HURC,FOR_COMPANY
2,2,HURC,FOR_COMPANY
3,3,HURC,FOR_COMPANY
4,4,HURC,FOR_COMPANY


In [28]:
full_for_company_df.to_csv("./data/full/for_company.csv", index=False)
incremental_for_company_df.to_csv("./data/incremental/for_company.csv", index=False)

# Copy files to import directory

The path below is the neo4j_home directory for my Neo4j desktop instance. To find the location of your neo4j_home, 
you can run this query.
```
CALL dbms.listConfig() YIELD name, value 
WHERE name = "server.directories.neo4j_home"
RETURN value
```

In [29]:
desktop_location = "/Users/nathansmith/Library/Application Support/Neo4j Desktop/"\
                   "Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986"
desktop_db_home_dir = Path(desktop_location)

In [30]:
full_import_dir = desktop_db_home_dir.joinpath("import/full")
incremental_import_dir = desktop_db_home_dir.joinpath("import/incremental")

In [31]:
full_import_dir.mkdir(parents=True, exist_ok=True)
incremental_import_dir.mkdir(parents=True, exist_ok=True)

In [32]:
os.system(f'cp ./data/full/*.csv "{full_import_dir}"')
os.system(f'cp ./data/incremental/*.csv "{incremental_import_dir}"')

0

# Execute full import

In [33]:
os.chdir(desktop_db_home_dir)

In case Neo4j is not running, start it up.

In [34]:
os.system("./bin/neo4j-admin server start")

Configuration file validation successful (with warnings).


Validating Neo4j configuration: /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/conf/neo4j.conf
5 issues found.

Validating user Log4j configuration: /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/conf/user-logs.xml
No issues found.

Validating server Log4j configuration: /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/conf/server-logs.xml
No issues found.

Neo4j is already running (pid:1317).
Run with '--verbose' for a more detailed error message.


256

Use getpass to avoid storing the password in the notebook in plain text.

In [35]:
password = getpass()

········


The database should not exist before doing a full import. If it does exist, drop it before executing the import script. This notebook uses cypher-shell to run Cypher queries through the command line. You could use the [Neo4j Python driver](https://neo4j.com/docs/api/python-driver/current/) to execute these queries, or you could run them in Neo4j Browser.

In [36]:
os.system(f'./bin/cypher-shell -u neo4j -p {password} "DROP DATABASE insiderdb"')

0

The command specifies that we are doing a full import. The last argment in the command is the name of the database that will be created.

In [37]:
command = ("./bin/neo4j-admin database import full "
           "--nodes import/full/company.csv --nodes import/full/owner.csv --nodes import/full/transaction.csv "
           "--relationships import/full/made_transaction.csv --relationships import/full/for_company.csv "
           "insiderdb")

In [38]:
print(command)

./bin/neo4j-admin database import full --nodes import/full/company.csv --nodes import/full/owner.csv --nodes import/full/transaction.csv --relationships import/full/made_transaction.csv --relationships import/full/for_company.csv insiderdb


In [39]:
os.system(command)

Neo4j version: 5.8.0
Importing the contents of these files into /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/data/databases/insiderdb:
Nodes:
  /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/import/full/company.csv
  /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/import/full/owner.csv
  /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/import/full/transaction.csv

Relationships:
  /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/import/full/made_transaction.csv
  /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-


Import starting 2023-05-25 08:26:29.166-0500
  Estimated number of nodes: 30.33 k
  Estimated number of node properties: 197.45 k
  Estimated number of relationships: 40.00 k
  Estimated number of relationship properties: 0.00 
  Estimated disk space usage: 6.342MiB
  Estimated required memory usage: 510.4MiB

(1/4) Node import 2023-05-25 08:26:30.426-0500
  Estimated number of nodes: 30.33 k
  Estimated disk space usage: 5.045MiB
  Estimated required memory usage: 510.4MiB
.......... .......... .......... .......... ..........   5% ∆209ms
.......... .......... .......... .......... ..........  10% ∆1ms
.........- .......... .......... .......... ..........  15% ∆117ms
.......... .......... .......... .......... ..........  20% ∆0ms
.......... .......... .......... .......... ..........  25% ∆1ms
.......... .......... .......... .......... ..........  30% ∆0ms
.......... .......... .......... .......... ..........  35% ∆0ms
.......... .......... .......... .......... ..........  40% ∆

0

# Validate full import

The data for the learndb database has been loaded in to the Neo4j data store, but you have to run create database to update the systemdb before you can use the database.

In [40]:
os.system(f'./bin/cypher-shell -u neo4j -p {password} "CREATE DATABASE insiderdb"')

0

In [41]:
os.system(f'./bin/cypher-shell -u neo4j -p {password} -d insiderdb '
          '"MATCH (n) RETURN labels(n) AS label, count(*) AS nodeCount"')

label, nodeCount
["Owner"], 7904
["Company"], 2421
["Transaction"], 20000


0

In [42]:
os.system(f'./bin/cypher-shell -u neo4j -p {password} -d insiderdb '
          '"MATCH ()-[r]->() RETURN type(r) AS type, count(*) AS relationshipCount"')

type, relationshipCount
"MADE_TRANSACTION", 20000
"FOR_COMPANY", 20000


0

# Execute incremental import

We need uniqueness constraints on the node properties before we can do an incremental import. These scripts create them. You could use a uniqueness constraint or a node key, since node key implies uniqueness.

In [43]:
key_constraints = ["CREATE CONSTRAINT company_node_key IF NOT EXISTS FOR (c:Company) "
                   "REQUIRE c.issuerTradingSymbol IS NODE KEY",
                   "CREATE CONSTRAINT owner_node_key IF NOT EXISTS FOR (o:Owner) "
                   "REQUIRE o.reportingOwnerName IS NODE KEY",
                   "CREATE CONSTRAINT transaction_node_key IF NOT EXISTS FOR (t:Transaction) "
                   "REQUIRE t.transactionId IS NODE KEY"]                   

In [44]:
for command in key_constraints:
    os.system(f'./bin/cypher-shell -u neo4j -p {password} -d insiderdb "{command}"')

The database must be stopped during the incremental import.

In [45]:
os.system(f'./bin/cypher-shell -u neo4j -p {password} "STOP DATABASE insiderdb WAIT"')

address, state, message, success
"localhost:7687", "CaughtUp", "caught up", TRUE


0

In this command, we specify that we are dong incremental. `--force` is a required argument for incremental updates. `--stage=all` says to perform all the incremental import stages at once. The database must be stopped to run in this mode. It is possible to run some stages of the incremental import with the database online in read-only mode. This might be helpful for very large incremental imports. See the [documentation](https://neo4j.com/docs/operations-manual/current/tools/neo4j-admin/neo4j-admin-import/#import-tool-incremental-examples) for more information.

In [46]:
command = ("./bin/neo4j-admin database import incremental --force --verbose " 
           "--stage=all "
           "--nodes=import/incremental/company.csv --nodes=import/incremental/owner.csv "
           "--nodes=import/incremental/transaction.csv "
           "--relationships=import/incremental/made_transaction.csv "
           "--relationships=import/incremental/for_company.csv "
           "insiderdb")

In [47]:
print(command)

./bin/neo4j-admin database import incremental --force --verbose --stage=all --nodes=import/incremental/company.csv --nodes=import/incremental/owner.csv --nodes=import/incremental/transaction.csv --relationships=import/incremental/made_transaction.csv --relationships=import/incremental/for_company.csv insiderdb


In [48]:
os.system(command)

Executing command line: /Library/Java/JavaVirtualMachines/temurin-17.jdk/Contents/Home/bin/java -cp /Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/plugins/*:/Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/conf/*:/Users/nathansmith/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-915b370f-4832-433d-b2d1-cea3f375a986/lib/* -XX:+UseParallelGC -XX:-OmitStackTraceInFastThrow -XX:+UnlockExperimentalVMOptions -XX:+TrustFinalNonStaticFields -XX:+DisableExplicitGC -Djdk.nio.maxCachedBufferSize=1024 -Dio.netty.tryReflectionSetAccessible=true -XX:+ExitOnOutOfMemoryError -Djdk.tls.ephemeralDHKeySize=2048 -XX:FlightRecorderOptions=stackdepth=256 -XX:+UnlockDiagnosticVMOptions -XX:+DebugNonSafepoints --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.bas

Nodes, started 2023-05-25 13:26:57.970+0000
[*Nodes:?? 510.0MiB---------------------------------------------------------------------------]10.0K ∆10.0K
Done in 370ms
Prepare node index, started 2023-05-25 13:26:58.967+0000
[*:510.0MiB-----------------------------------------------------------------------------------]10.0K ∆10.0K
Done in 185ms
Relationships, started 2023-05-25 13:26:59.199+0000
[*Relationships:?? 510.0MiB-------------------------------------------------------------------]20.0K ∆20.0K
Done in 171ms
Node Degrees, started 2023-05-25 13:26:59.637+0000
[*>---------------------------------------------------|CALCULATE:510.4MiB---------------------] 120K ∆ 120K
Done in 26ms
Relationship --> Relationship 1-2/2, started 2023-05-25 13:26:59.694+0000
[>-----------|*LINK----------------------------------------------------------|v:??------------] 120K ∆ 120K
Done in 51ms
RelationshipGroup 1-2/2, started 2023-05-25 13:26:59.749+0000
[*>-------------------------------------------------

0

# Validate incremental import

Restart the database after the import.

In [49]:
os.system(f'./bin/cypher-shell -u neo4j -p {password} "START DATABASE insiderdb WAIT"')

address, state, message, success
"localhost:7687", "CaughtUp", "caught up", TRUE


0

We should see 30,000 transaction nodes.

In [50]:
os.system(f'./bin/cypher-shell -u neo4j -p {password} -d insiderdb '
          '"MATCH (n) RETURN labels(n) AS label, count(*) AS nodeCount"')

label, nodeCount
["Owner"], 10716
["Company"], 2928
["Transaction"], 30000


0

We should see 30,000 of each type of relationship.

In [51]:
os.system(f'./bin/cypher-shell -u neo4j -p {password} -d insiderdb '
          '"MATCH ()-[r]->() RETURN type(r) AS type, count(*) AS relationshipCount"')

type, relationshipCount
"MADE_TRANSACTION", 30000
"FOR_COMPANY", 30000


0